In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import os, os.path as osp
import numpy as np
import torch
import torch.nn as nn
import scipy.io

import matplotlib.pyplot as plt

In [ ]:
import alpine

In [ ]:
datafile = "./data/hyperspectral/icvl.mat"


In [ ]:
matdata = scipy.io.loadmat(datafile)

cube, wvls = matdata["hypercube"], matdata["wavelengths"]

In [ ]:
cube.min(), cube.max()

In [ ]:
cube = np.float32(cube)
normalized_cube = ((cube) - cube.min()) / (cube.max() - cube.min())
print(normalized_cube.min(), normalized_cube.max())

In [ ]:
print(cube.shape)
print(wvls.shape)
print(normalized_cube.shape)

In [ ]:
H, W, lmda = cube.shape
lmdaT = 52
coords = alpine.utils.get_coords_spatial(H, W, lmdaT)[None,...].float().cuda()
print(coords.shape)

In [ ]:
# dataset = alpine.dataloaders.BatchedNDSignalLoader(signal = normalized_cube, grid_dims = [H,W,lmda], normalize_signal = False)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size = int(1e3), shuffle = False, num_workers = 16, pin_memory = True)

In [ ]:
hyperspectral_compression = alpine.models.Siren(
    in_features=3,
    out_features=1,
    hidden_layers = 4,
    hidden_features = 256,
    outermost_linear = True,
).float().cuda()

In [ ]:
EPOCHS = 5000

In [ ]:
fit_spectrum = hyperspectral_compression.fit_signal(dataloader=dataloader, enable_tqdm=True, n_iters = EPOCHS)  

In [ ]:
output = hyperspectral_compression.render(coords[...,100,:])

In [ ]:
plt.figure()
plt.imshow(output['output'][0].cpu().detach().numpy(), cmap="gray")